In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from Output_Processor import trajectory_plots

In [14]:
df = pd.read_csv('out/compare/benchmark full route/numer_results.csv')
df['prc_red_eh'] = (-df['EH']+df['NC'])/(df['NC'])*100
df['prc_red_ehx'] = (-df['EHX']+df['NC'])/(df['NC'])*100
df['prc_red_eh'] = df['prc_red_eh'].round(decimals=2)
df['prc_red_ehx'] = df['prc_red_ehx'].round(decimals=2)
df[['parameter', 'prc_cancel', 'prc_red_eh', 'prc_red_ehx']]

,parameter,prc_cancel,prc_red_eh,prc_red_ehx
0,cv_h,0,-1.21,-1.96
1,cv_h,15,13.75,24.69
2,cv_h,30,17.56,18.24
3,wt,0,-2.52,-3.60
4,wt,15,7.91,13.82
5,wt,30,12.69,14.17
6,rbt,0,-1.54,0.38
7,rbt,15,13.94,23.45
8,rbt,30,14.92,18.71
9,lp,0,5.41,-2.70


In [15]:
df = pd.read_csv('out/compare/benchmark mid route/numer_results.csv')
df['prc_red_eh'] = (-df['EH']+df['NC'])/(df['NC'])*100
df['prc_red_ehx'] = (-df['EHX']+df['NC'])/(df['NC'])*100
df['prc_red_eh'] = df['prc_red_eh'].round(decimals=2)
df['prc_red_ehx'] = df['prc_red_ehx'].round(decimals=2)
df[['parameter', 'prc_cancel','prc_red_eh', 'prc_red_ehx']]

,parameter,prc_cancel,prc_red_eh,prc_red_ehx
0,cv_h,0,1.04,-2.17
1,cv_h,15,17.11,29.67
2,cv_h,30,21.20,20.28
3,wt,0,-1.13,-1.77
4,wt,15,7.00,12.52
5,wt,30,16.89,15.35
6,rbt,0,-1.70,-0.36
7,rbt,15,15.21,23.35
8,rbt,30,19.57,21.23
9,lp,0,0.00,3.57


In [17]:
df = pd.read_pickle('out/EHX_dispatch/0714-120640-trip_record_ob.pkl')

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/Users/joseph/opt/anaconda3/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-darwin.so'>

In [18]:
pd. __version__

'1.2.4'